In [2]:
%load_ext autoreload
%autoreload 1 
%aimport ncaa_pbp

import requests
from bs4 import BeautifulSoup
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
base_url = "https://stats.ncaa.org/"
import pandas as pd
import numpy as np
import re
from io import StringIO
from copy import deepcopy
from itertools import combinations
from collections import defaultdict, Counter
import json

from ncaa_pbp import Game, Team, get_Teams

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
a = Team('561062',source='file')
a.roster

,PlayerID,Name,#,Class,Position,Height,Hometown,High School,GP,GS
0,8161193,Isabel Tillett,10,So.,G,5-9,"North Chesterfield, VA",Cosby,13,0
1,8161194,Bri Johns,35,Sr.,G,5-11,"Alexandria, VA",Thomas Edison,23,2
2,8161198,Kennedy Calhoun,4,Jr.,G,5-5,"Harrisburg, NC",Hickory Ridge,40,40
3,8161199,Ashleigh Connor,15,So.,G,5-11,"Pittsburgh, PA",Mount Lebanon,18,0
4,8161200,Mia Nicastro,22,So.,F,6-2,"Saint Charles, MO",St. Charlies West,26,0
5,8161202,Peyton Kennedy,32,Sr.,F,5-11,"Rockford, IL",Boylan Catholic,40,40
6,8381735,Camreé Clegg,2,Sr.,G,5-5,"Westland, MI",Wayne Memorial,40,0
7,8381736,Nafatoumata Haidara,21,Sr.,F,6-5,"Bamako, Mali",Largo,0,0
8,8381743,Kyla McMakin,00,Sr.,G,5-11,"Virginia Beach, VA",Ocean Lakes,40,40
9,8381758,Brooklyn Gray,45,So.,G,5-11,"Rockford, IL",Auburn,37,23


In [44]:
r = requests.get(base_url + f'contests/{'3970485'}/individual_stats', headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')
teams = [team['href'].split('/')[-1] for team in soup.find('table').find_all('a', {'class':'skipMask'}) if team.text]
#self.teams = [teams[0], teams[2]]

tables = pd.read_html(StringIO(r.text),match='FGM',header=0)
boxScore = {teams[1]: tables[1], teams[0]: tables[0]}

# Properly format starters' names for each team
starters = {team:[" ".join(player.split(', ')[::-1]) for player in boxScore[team].loc[boxScore[team].GS==1,'Name'].tolist()[:-1]] \
                    for team in teams}
starters

{'560720': ['Courtney Becker',
  'Grace Berg',
  'Katie Dinnebier',
  'Taylor McAulay',
  'Anna Miller'],
 '561062': ['Kennedy Calhoun',
  'Peyton Kennedy',
  'Julia Martinez',
  'Kyla McMakin',
  'Marcavia Shavers']}

In [49]:
soup = BeautifulSoup(html_file['box'], 'html.parser')
teams = [team['href'].split('/')[-1] for team in soup.find('table').find_all('a', {'class':'skipMask'}) if team.text]
#self.teams = [teams[0], teams[2]]

tables = pd.read_html(StringIO(r.text),match='FGM',header=0)
boxScore = {teams[1]: tables[1], teams[0]: tables[0]}

# Properly format starters' names for each team
starters = {team:[" ".join(player.split(', ')[::-1]) for player in boxScore[team].loc[boxScore[team].GS==1,'Name'].tolist()[:-1]] \
                    for team in teams}
starters

{'560720': ['Courtney Becker',
  'Grace Berg',
  'Katie Dinnebier',
  'Taylor McAulay',
  'Anna Miller'],
 '561062': ['Kennedy Calhoun',
  'Peyton Kennedy',
  'Julia Martinez',
  'Kyla McMakin',
  'Marcavia Shavers']}

In [46]:
with open("json_file.json", mode="w", encoding="utf-8") as jsonf:
    json.dump(x,jsonf)

In [47]:
with open("json_file.json", mode='r', encoding='utf-8') as htmlf:
    html_file = json.load(htmlf)

In [41]:
a = Game("3978635",loadShotChart=True,source='web')
z = a.pbp

In [42]:
a.shotChart

,cx,cy,id,Period,player_id,team_id,ShotMade,Team,Player,Time,game_id
0,65.80000000000001,280,play_2573621660,period_1,player_768905434,team_248,made,George Mason,Paula Suarez,09:50:00,3978635
1,84.6,295,play_2573621662,period_1,player_768905431,team_248,missed,George Mason,Jazmyn Doster,08:56:00,3978635
2,827.2,290,play_2573621664,period_1,player_768905454,team_609,missed,Saint Louis,Julia Martinez,08:50:00,3978635
3,56.4,260,play_2573621667,period_1,player_768905434,team_248,made,George Mason,Paula Suarez,08:42:00,3978635
4,122.2,330,play_2573621669,period_1,player_768905446,team_248,made,George Mason,Ta&#39;Viyanna Habib,08:05:00,3978635
...,...,...,...,...,...,...,...,...,...,...,...
116,329,215,play_2573622739,period_4,player_768905469,team_609,missed,Saint Louis,Tierra Simon,01:31:00,3978635
117,705,295,play_2573622743,period_4,player_768905444,team_248,missed,George Mason,Kennedy Harris,01:18:00,3978635
118,94,290,play_2573622747,period_4,player_768905464,team_609,missed,Saint Louis,Kennedy Calhoun,01:11:00,3978635
119,855.4,150,play_2573622776,period_4,player_768905436,team_248,missed,George Mason,Jada Brown,00:24:60,3978635


In [48]:
import os
os.path.isdir('DataDirectory/Team')

True

In [64]:
x=df.iloc[np.where(df.eventType==6)[0]-1]
x.loc[x.eventType!=3]

,Quarter,Time,561062,Score,560762,text,team,eventType
82,1,02:18:00,NaN,7-15,"Kennedy Harris, freethrow 1of2 missed","Kennedy Harris, freethrow 1of2 missed",560762,13
183,2,05:59:00,NaN,17-31,"Team, rebound offensive team;","Team, rebound offensive team;",560762,6
184,2,05:59:00,NaN,17-31,"Jazmyn Doster, rebound defensive","Jazmyn Doster, rebound defensive",560762,6
192,2,04:59:00,NaN,20-31,"Sonia Smith, substitution in","Sonia Smith, substitution in",560762,12
261,2,00:48:80,NaN,28-45,"Zahirah Walton, 2pt layup 2ndchance;pointsinth...","Zahirah Walton, 2pt layup 2ndchance;pointsinth...",560762,4
274,2,00:00:40,"Tierra Simon, 2pt layup 2ndchance;pointsinthep...",32-47,NaN,"Tierra Simon, 2pt layup 2ndchance;pointsinthep...",561062,4
433,4,09:29:00,"Julia Martinez, freethrow 1of2 made",44-74,NaN,"Julia Martinez, freethrow 1of2 made",561062,13
479,4,06:39:00,"Marcavia Shavers, 2pt layup 2ndchance;pointsin...",49-79,NaN,"Marcavia Shavers, 2pt layup 2ndchance;pointsin...",561062,4
512,4,04:04:00,NaN,52-83,"Zahirah Walton, 2pt layup 2ndchance;pointsinth...","Zahirah Walton, 2pt layup 2ndchance;pointsinth...",560762,4
554,4,01:02:00,NaN,58-91,"Taylor Jameson, freethrow 1of2 made","Taylor Jameson, freethrow 1of2 made",560762,13


In [6]:
class Possession():

    def __init__(self, events):
        self.quarter = events[0].quarter
        self.events = events

    @property
    def team(self):
        return self.__OffenseTeam
    
    @property
    def startTime(self):
        return self.__time
    
    @property
    def endTime(self):
        return self.events[-1].time
    
    @property
    def quarter(self):
        return self.__quarter
    
    @property
    def previousEvent(self):
        return self.__previousEvent
    
    @previousEvent.setter
    def previousEvent(self, event):
        self.__previousEvent = event
    
    @property
    def currentLineups(self):
        pass
    
    @currentLineups.setter
    def currentLineups(self, lineups):
        self.__currentLineups = lineups
        
    @property
    def data(self):
        return self.__dict__

class PossessionLoader():

    def __init__(self, pbp) -> None:
        self.load_poss(pbp)
    
    def load_poss(self, pbp):
        endingIndices = np.where([item.isPossessionEnding for item in pbp.items])[0]+1
        possessions = np.split(z.items,endingIndices)

        # Create list of possession items
        self.items = [Possession(poss) for poss in possessions]

In [9]:
teams = a.teams
teams

['561062', '560762']

In [4]:
ends = np.where([item.isPossessionEnding for item in z.items])[0]+1
poss = np.split(z.items,ends)

In [77]:
num = 3
p = poss[num]
for event in poss[num]:
    print(event.description, event.team)
    


Jazmyn Doster, block 560762
Julia Martinez, 2pt jumpshot pointsinthepaint;blocked; missed 561062
Paula Suarez, rebound defensive 560762


In [78]:
x = {team:[e.eventType if e.team==team else -1*e.eventType if e.team else None for e in p ] for team in teams}
x

{'561062': [-2, 3, -6], '560762': [2, -3, 6]}

In [37]:
list(combinations(p[-1].currentLineups[teams[0]],1))

[('Kennedy Calhoun', 'Peyton Kennedy', 'Julia Martinez', 'Kyla McMakin'),
 ('Kennedy Calhoun', 'Peyton Kennedy', 'Julia Martinez', 'Marcavia Shavers'),
 ('Kennedy Calhoun', 'Peyton Kennedy', 'Kyla McMakin', 'Marcavia Shavers'),
 ('Kennedy Calhoun', 'Julia Martinez', 'Kyla McMakin', 'Marcavia Shavers'),
 ('Peyton Kennedy', 'Julia Martinez', 'Kyla McMakin', 'Marcavia Shavers')]

In [114]:
lineupData = defaultdict(lambda: defaultdict(int))
for p in poss:
    # Event occurances for each team during the possession
    x = {team:[e.eventType if e.team==team else -1*e.eventType if e.team else None for e in p ] for team in teams}
    for team in teams:
        # Count the number of occurances for each event type while lineups where on the floor
        for lineup in list(combinations(p[-1].currentLineups[team],3)):
            for pt in x[team]:
                lineupData[lineup][pt] += 1

dict(lineupData)

{('Kennedy Calhoun',
  'Peyton Kennedy',
  'Julia Martinez'): defaultdict(int,
             {None: 3,
              1: 1,
              -1: 1,
              -4: 7,
              10: 6,
              -3: 3,
              6: 3,
              -2: 1,
              3: 4,
              -6: 4,
              -9: 2,
              -5: 2,
              11: 1,
              12: 2,
              4: 2,
              5: 1,
              -7: 5,
              7: 5,
              -10: 3,
              8: 1,
              9: 1}),
 ('Kennedy Calhoun',
  'Peyton Kennedy',
  'Kyla McMakin'): defaultdict(int,
             {None: 3,
              1: 1,
              -1: 1,
              -4: 7,
              10: 7,
              -3: 4,
              6: 4,
              -2: 1,
              3: 4,
              -6: 4,
              -9: 3,
              -5: 2,
              11: 1,
              12: 4,
              4: 3,
              5: 1,
              -7: 6,
              7: 6,
              -10: 4,
          

In [101]:
{key:Counter(v) for key,v in dict(lineupData).items()}

{('Kennedy Calhoun',): Counter({12: 36,
          -12: 32,
          7: 30,
          -7: 28,
          -6: 25,
          3: 23,
          6: 21,
          -3: 20,
          -4: 19,
          10: 14,
          -10: 13,
          -5: 11,
          -13: 11,
          4: 10,
          None: 6,
          -9: 6,
          5: 6,
          13: 6,
          -2: 3,
          8: 3,
          9: 3,
          1: 1,
          -1: 1,
          11: 1,
          2: 1}),
 ('Peyton Kennedy',): Counter({12: 38,
          -12: 34,
          -6: 25,
          -7: 25,
          7: 25,
          3: 23,
          -4: 22,
          -3: 19,
          6: 17,
          -10: 13,
          4: 12,
          10: 11,
          -5: 9,
          5: 9,
          13: 8,
          None: 7,
          -9: 5,
          -13: 5,
          -2: 3,
          8: 2,
          9: 2,
          2: 2,
          1: 1,
          -1: 1,
          11: 1}),
 ('Julia Martinez',): Counter({12: 50,
          -12: 38,
          7: 34,
          

In [109]:
y = defaultdict(lambda: defaultdict(int))
y['a']['a'] +=1
dict(y)

{'a': defaultdict(int, {'a': 1})}

In [60]:
Counter([1,1,1,1])

Counter({1: 4})

In [48]:
list(map(lambda x: combinations(x[-1].currentLineups,3), poss))

In [19]:
li =[1,2,3]
li[3] = 5

IndexError: list assignment index out of range

In [89]:
for p in poss:
    for item in p:
        print(item.description)
    print('------')

game start
period start
jumpball startperiod
Marcavia Shavers, jumpball lost
Jazmyn Doster, jumpball won
Paula Suarez, 2pt layup pointsinthepaint; made
------
Team, turnover shotclock team;
------
Jazmyn Doster, 2pt jumpshot fromturnover;pointsinthepaint; missed
Julia Martinez, rebound defensive
------
Jazmyn Doster, block
Julia Martinez, 2pt jumpshot pointsinthepaint;blocked; missed
Paula Suarez, rebound defensive
------
Paula Suarez, 2pt layup pointsinthepaint;fastbreak; made
------
Peyton Kennedy, turnover badpass
------
Ta'Viyanna Habib, 2pt jumpshot fromturnover; made
------
Paula Suarez, steal
Kyla McMakin, turnover lostball
------
Ta'Viyanna Habib, 2pt floatingjumpshot fromturnover;pointsinthepaint; made
------
Paula Suarez, assist
Team, timeout short
Marcavia Shavers, substitution out
Tierra Simon, substitution in
Tierra Simon, 2pt jumpshot missed
Ta'Viyanna Habib, rebound defensive
------
Sonia Smith, 2pt layup pointsinthepaint; missed
Team, rebound offensive team;
Sonia Smith